In [1]:
%%time
import pandas as pd
import matplotlib.pyplot as plt
from read_data import df
%load_ext autoreload
%autoreload 2

In [2]:
# df = df[:5] # short version for test
df

убираем сложные поля утреннего и вечернего отчета, переименовываем вложенные поля, добиваемся плоской структуры данных

In [3]:
%%time
dfm = df['morning'].apply(pd.Series)
dfe = df['evening'].apply(pd.Series)
dfm.columns = 'm_' + dfm.columns
dfe.columns = 'e_' + dfe.columns

df = pd.concat([df.drop(['morning', 'evening'], axis=1),dfm, dfe], axis=1)
print(df.shape)
# df

In [4]:
df.dropna
print(df.shape)

In [5]:
df.dtypes

добавляем дополнительные столбцы с необходимыми данными

In [6]:
%%time
df['income'] = df['e_cash']+df['e_cashless']
df['date'] = pd.to_datetime(df.year*10000+df.month*100+df.day,format='%Y%m%d')

In [7]:
%%time
df = df.set_index(['date'])

In [8]:
import numpy as np
print(np.sort(df.m_user.unique()))
print(len(df.m_user.unique()))

In [9]:
df.m_user = df.m_user.str.strip().str.title()
for i in np.sort(df.m_user.unique()):
    print(i)
print(len(df.m_user.unique()))


In [10]:
%%time
# based on stackoverflow patricksurry answer https://stackoverflow.com/a/27802006
# grouped = (df
#     .groupby('store')                
#     .apply(lambda g:              
#         g[['income']]
#         .resample('W').mean()  # change resample argument
#         )
#     .unstack(level=0)  # pivot the store index rows to columns
#     # .fillna(0)
# )
# grouped.columns=grouped.columns.droplevel()
# grouped.plot(figsize=(10,10), title='усредненная по неделям выручка в разрезе тиров')


In [70]:
%%time
df.groupby(['store']).resample('W').mean()['income'].unstack(level=0)\
    .plot(legend=True, figsize=(10,10), title='усредненная по неделям выручка в разрезе тиров')

In [46]:
%%time
df[(df.m_user == 'Юля') | (df.m_user == 'Юля Халикова')]\
    .groupby(['m_user'])['income']\
    .plot(legend=True, figsize=(10,10), title='tests', style='o-')


In [12]:
%%time
df.groupby(['m_user']).resample('W').mean()['income'].unstack(level=0)\
    .plot(legend=True, figsize=(10,10), title='усредненная по неделям выручка в разрезе операторов')

In [18]:
%%time
df.groupby(['m_user']).count()['income']\
    .plot(kind='bar', figsize=(10,10), title='кол-во записей о работе каждого оператора')

In [24]:
%%time
print(f"среднее кол-во записей про отдельного оператора: {df.groupby(['m_user']).count()['income'].mean():.1f}")
print(f"медиана кол-ва записей про отдельного оператора: {df.groupby(['m_user']).count()['income'].median():.1f}")

средняя выручка в разрезе точек и операторов

In [87]:
df.groupby(['store', 'm_user'])['income'].mean()



